# purpose and use notes

purpose: generates NPS scores for traits used in this paper. If overwrite_file==TRUE, the network propagation scores will be overwritten with the newly generated scores. This will affect all downstream analyses.

runs network propagation (typically in pcnet v1.4) from seed genes saved in a file- accessed from meta data csv. if rerun==TRUE, then network propagation scores will be recalculated.

# setup

In [3]:
#read in libraries
#from rca_functions import *
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import matplotlib
from matplotlib_venn import venn2 
from scipy.stats import hypergeom
import statsmodels.stats.multitest
import rca_functions
import ndex2
import networkx as nx
from netcoloc import netprop_zscore
from netcoloc import netprop
from netcoloc import network_colocalization
import sys

In [4]:
os.chdir('../')

In [5]:
os.getcwd()

'/tscc/projects/ps-palmer/brittany/rare_common_alcohol_comparison'

In [6]:
save_file=False

# Interactome Set-up

In [7]:
UUIDs= {
    'pcnet_v14':'c3554b4e-8c81-11ed-a157-005056ae23aa',
    'pcnet_v13':'4de852d9-9908-11e9-bcaf-0ac135e8bacf',
    'string':'98ba6a19-586e-11e7-8f50-0ac135e8bacf',
    'humanNet_v3_FN': '40913318-3a9c-11ed-ac45-0ac135e8bacf',
    'ACN':'29b2d215-07fd-11ef-9621-005056ae23aa',
	'ACN_unannot':'f81a3f67-4215-11ee-aa50-005056ae23aa',
	'ACN_strin':'48de252c-3d50-11ee-aa50-005056ae23aa',
}

def import_interactome(interactome_name=None, ndex_user=None, ndex_password=None,UUID=None):
    interactome_uuid=UUIDs[interactome_name]
    print(interactome_name)
    ndex_server='public.ndexbio.org'
    #import network based on provided interactome key
    if (interactome_name in UUIDs.keys()):
        graph = ndex2.create_nice_cx_from_server(
                    ndex_server, 
                    username=ndex_user, 
                    password=ndex_password, 
                    uuid=interactome_uuid
                ).to_networkx()
        if (interactome_name=='pcnet_v14'):
            graph=nx.relabel_nodes(graph, nx.get_node_attributes(graph, 'HGNC Symbol'))
        # print out interactome num nodes and edges for diagnostic purposes
        print('number of nodes:')
        print(len(graph.nodes))
        print('\nnumber of edges:')
        print(len(graph.edges))
        return(graph)
    elif(interactome_name==None & UUID!=None):
        print('using novel UUID. For UUIDs used in this study, see UUID_dict')
        graph = ndex2.create_nice_cx_from_server(
            ndex_server, 
            username=ndex_user, 
            password=ndex_password, 
            uuid=UUID
        ).to_networkx()
        # print out interactome num nodes and edges for diagnostic purposes
        print('number of nodes:')
        print(len(graph.nodes))
        print('\nnumber of edges:')
        print(len(graph.edges))
        return(graph)
    else:
        print('UUID/interactome name not provided- please provide either to import interactome.')
    #relabel the nodes with the gene name instead of an arbitrary number


In [8]:
interactome_name='pcnet_v14'

In [9]:
graph=import_interactome(interactome_name)

pcnet_v14
number of nodes:
18630

number of edges:
2687393


In [ ]:
# pre calculate the matricies used for network propagation
print('\ncalculating w_prime')
w_prime = netprop.get_normalized_adjacency_matrix(graph, conserve_heat=True)

print('\ncalculating w_double_prime')
w_double_prime = netprop.get_individual_heats_matrix(w_prime, .5)


calculating w_prime

calculating w_double_prime


# common gene data analysis

In [ ]:
def import_seedgenes(path,pcol='P',gene_col='GENE NAME',delim='comma', cutoff=None):
    if delim=='comma':
        df=pd.read_csv(path,sep=',')
    else:
        df=pd.read_csv(path,sep='\t')
    if pcol==None:
        print('pvalue column not specified- all genes will be used')
        cutoff=None
    if cutoff=='bonferroni':
        df=df[df[pcol]<0.05/len(df)]
    elif cutoff=='FDR':
        df['pval_FDR']=statsmodels.stats.multitest.fdrcorrection(df[pcol],alpha=0.05,method='indep',is_sorted=False)[1]
        df=df[df['pval_FDR']<0.05]
    else:
        print('cutoff not defined/custom- using all genes ')
        df=df
    print(df.head())
    #gene_ls=list(set(df[gene_col]))
    #return(gene_ls)
    return(df)

In [ ]:
common_datasets=pd.read_csv('common_datasets_prepub.csv',sep=',')
rare_datasets=pd.read_csv('rare_datasets_prepub.csv',sep=',')

In [ ]:
datasets=rare_datasets
row=0

In [ ]:
#import seed genes- rare
seed_genes = set(import_seedgenes(datasets['seed_path'][row], 
                                  None, 
                                  datasets['seed_gene_name'][row], 
                                  datasets['delim'][row])[datasets['seed_gene_name'][row]])
#filter for only genes in the interactome
seed_genes = list(seed_genes.intersection(graph.nodes()))

In [ ]:
z_score, Fnew_score, Fnew_rand_score = netprop_zscore.calculate_heat_zscores(
    w_double_prime,  
    list(graph.nodes()),
    dict(graph.degree), 
    seed_genes, num_reps=1000,
    minimum_bin_size=100)

In [ ]:
#import seed genes
seed_genes = set(import_seedgenes(datasets['seed_path'][row_common], 
                                  datasets['seed_p'][row_common], 
                                  datasets['seed_gene_name'][row_common], 
                                  datasets['delim'][row_common])[datasets['seed_gene_name'][row_common]])
#filter for only genes in the interactome
seed_genes = list(seed_genes.intersection(graph.nodes()))
z_score, Fnew_score, Fnew_rand_score = netprop_zscore.calculate_heat_zscores(
    w_double_prime,  
    list(graph.nodes()),
    dict(graph.degree), 
    seed_genes, num_reps=1000,
    minimum_bin_size=100)


In [133]:
datasets.columns

Index(['label', 'cutoff used', 'seed_path', 'delim', 'zscore_file',
       'zscore_path', 'seed_gene_name', 'phenotype_group'],
      dtype='object')

In [142]:
seed_genes

['FOXP1',
 'MMEL1',
 'SLC29A1',
 'ZNF449',
 'ANXA3',
 'ADH1C',
 'UBR3',
 'OR51D1',
 'ANKRD12',
 'CHRM3',
 'GIGYF1',
 'ADH1A',
 'PPFIA3',
 'AKAP6',
 'SMG1',
 'MTOR',
 'SCN7A',
 'AKAP9',
 'FRK',
 'KIF21B',
 'DNAJA4',
 'KDM5B',
 'GET4',
 'TSPYL2',
 'ADH1B',
 'GRM5',
 'STK31',
 'ZPR1',
 'PMM2',
 'PLIN5',
 'ATP2B1',
 'CPSF6']

In [ ]:
z_score, Fnew_score, Fnew_rand_score = netprop_zscore.calculate_heat_zscores(
    w_double_prime,  
    list(graph.nodes()),
    dict(graph.degree), 
    seed_genes, num_reps=1000,
    minimum_bin_size=100)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
#import seed genes
seed_genes = set(import_seedgenes(datasets['seed_path'][row], 
                                  datasets['seed_p'][row], 
                                  datasets['seed_gene_name'][row], 
                                  datasets['delim'][row_common])[datasets['seed_gene_name'][row]])
#filter for only genes in the interactome
seed_genes = list(seed_genes.intersection(graph.nodes()))
z_score, Fnew_score, Fnew_rand_score = netprop_zscore.calculate_heat_zscores(
    w_double_prime,  
    list(graph.nodes()),
    dict(graph.degree), 
    seed_genes, num_reps=1000,
    minimum_bin_size=100)


In [ ]:
def run_net_prop(path, trait_name, pcol, gene_col, delim, cutoff=None, graph=None, w_double_prime=None, interactome='pcnet_v14', ndex_user=None, ndex_password=None, savefile=False):
    graph_nodes = list(graph.nodes())
    #print(graph_nodes)
    data = list(set(data).intersection(graph_nodes))
    #print(data)
    ##calculate heats
    z_score, Fnew_score, Fnew_rand_score = netprop_zscore.calculate_heat_zscores(
        w_double_prime,  
        graph_nodes,
        dict(graph.degree), 
        data, num_reps=1000,
        minimum_bin_size=100
    )
    if savefile:
        export_path = 'calculated_values/network_scores/'
        if graph is None and interactome == 'pcnet_v14':
            prefix = (export_path + trait_name).lower()
        elif graph is None and interactome != 'pcnet_v14':
            prefix = (export_path + trait_name + '_' + interactome).lower()
        elif graph is not None and interactome != 'pcnet_v14':
            prefix = (export_path + trait_name + '_' + interactome).lower()
        else:
            print("saving file without interactome_prefix, please provide an interactome name if prefix wanted")
            prefix = ('network_scores/' + trait_name).lower()

        z_score.to_csv(prefix + '_zscore.tsv', sep='\t', header=False)
        if saveheat:
            Fnew_score.to_csv(prefix + '_heats.tsv', sep='\t', header=False)
            pd.DataFrame(Fnew_rand_score, columns=z_score.index).to_csv((prefix+'_randheats.tsv'),sep='\t')
        else:
            print('calculated NPS not saved')
    return z_score

In [ ]:
data = import_seedgenes(path, pcol, gene_col, delim)
data = list(data[gene_col])
if graph is None:
    graph = import_interactome(interactome)
    print("importing network " + interactome)
if w_double_prime is None:
    # pre calculate mats used for netprop
    print('\ncalculating w_prime')
    w_prime = netprop.get_normalized_adjacency_matrix(graph, conserve_heat=True) 
    print('\ncalculating w_double_prime')
    w_double_prime = netprop.get_individual_heats_matrix(w_prime, 0.5)
else:
    print

In [30]:
str(delim)

"('comma',)"

In [19]:
z_score, Fnew_score, Fnew_rand_score = netprop_zscore.calculate_heat_zscores(
    w_double_prime,  
    graph_nodes,
    dict(graph.degree), 
    data, num_reps=1000,
    minimum_bin_size=100
)

NameError: name 'graph_nodes' is not defined

In [ ]:
for row_common in range(len(datasets)):
    print('processing '+datasets['label'][row_common])
    NPSc=run_net_prop(
        path=datasets['seed_path'][row_common], 
                 trait_name=datasets['label'][row_common],
                 pcol=datasets['seed_p'][row_common],
                 gene_col=datasets['seed_gene_name'][row_common],
                 delim=datasets['delim'][row_common],
                 cutoff=datasets['cutoff'][row_common],
                 graph=interactome,
                 interactome='pcnet_v14',
                 w_double_prime=w_double_prime, 
                 savefile=False)    
    print(NPSc.head)

In [ ]:
zscores

In [35]:
datasets['cutoff'][row_common]

'bonferroni'

In [15]:
def run_net_prop(path, trait_name, pcol, gene_col, delim, cutoff=None, graph=None, w_double_prime=None, interactome='pcnet_v14', ndex_user=None, ndex_password=None, savefile=False):
    """
    Executes network propagation analysis for a given trait using provided seed genes and provided interactome.

    Parameters:
    - path (str): The file path to the seed gene file.
    - trait_name (str): The name of the trait for which the analysis is being run.
    - pcol (str): The column name in the seed genes file that contains the p-values.
    - gene_col (str): The column name in the seed genes file that specifies the gene names.
    - delim (str): The delimiter used in the seed genes file.
    - cutoff (float, optional): The p-value cutoff for filtering seed genes. If None (Default), no filtering is applied. Defaults to None.
    - graph (NetworkX graph, optional): The interactome network graph. If None, the graph is imported using the interactome parameter. Defaults to None.
    - w_double_prime (numpy.ndarray, optional): Pre-calculated matrix for network propagation. If None, it is calculated in the function. Defaults to None.
    - interactome (str, optional): The name of the interactome. If no graph is provided, this will be imported using the import_interactome function which accepts UUIDs or keys to the UUIDs dictionary. Will used as a label for exported interactome files. Defaults to 'pcnet_v14', which was used for this analysis.
    - ndex_user (str, optional): NDEx account username, required if uploading results to NDEx. Defaults to None.
    - ndex_password (str, optional): NDEx account password, required if uploading results to NDEx. Defaults to None.

    Returns:
	NPS zscores
 
    Notes:
    - The function requires an external library for network propagation calculations.
    - The seed genes file should contain a column for genes and a column for their associated p-values.
    - The function saves three files: z-scores, raw heats, and randomized heats for the network analysis,
      with the trait name and optionally the interactome name as part of the filenames.
    - If using a private interactome, ensure the ndex_user and ndex_password are correctly provided.
    """
    data = import_seedgenes(path, pcol, gene_col, delim)
    data = list(data[gene_col])
    if graph is None:
        graph = import_interactome(interactome)
        print("importing network " + interactome)
    if w_double_prime is None:
        # pre calculate mats used for netprop
        print('\ncalculating w_prime')
        w_prime = netprop.get_normalized_adjacency_matrix(graph, conserve_heat=True) 
        print('\ncalculating w_double_prime')
        w_double_prime = netprop.get_individual_heats_matrix(w_prime, 0.5)
    else:
        print("using provided w_double_prime - please ensure that w_double_prime aligns to graph provided")
    graph_nodes = list(graph.nodes())
    #print(graph_nodes)
    data = list(set(data).intersection(graph_nodes))
    #print(data)
    ##calculate heats
    z_score, Fnew_score, Fnew_rand_score = netprop_zscore.calculate_heat_zscores(
        w_double_prime,  
        graph_nodes,
        dict(graph.degree), 
        data, num_reps=1000,
        minimum_bin_size=100
    )
    if savefile:
        export_path = 'calculated_values/network_scores/'
        if graph is None and interactome == 'pcnet_v14':
            prefix = (export_path + trait_name).lower()
        elif graph is None and interactome != 'pcnet_v14':
            prefix = (export_path + trait_name + '_' + interactome).lower()
        elif graph is not None and interactome != 'pcnet_v14':
            prefix = (export_path + trait_name + '_' + interactome).lower()
        else:
            print("saving file without interactome_prefix, please provide an interactome name if prefix wanted")
            prefix = ('network_scores/' + trait_name).lower()

        z_score.to_csv(prefix + '_zscore.tsv', sep='\t', header=False)
        if saveheat:
            Fnew_score.to_csv(prefix + '_heats.tsv', sep='\t', header=False)
            pd.DataFrame(Fnew_rand_score, columns=z_score.index).to_csv((prefix+'_randheats.tsv'),sep='\t')
        else:
            print('calculated NPS not saved')
    return z_score

In [16]:
def import_seedgenes(path,pcol='P',gene_col='GENE NAME',delim='comma', cutoff=None):
    if delim=='comma':
        df=pd.read_csv(path,sep=',')
    else:
        df=pd.read_csv(path,sep='\t')
    if pcol==None:
        print('pvalue column not specified- all genes will be used')
        cutoff=None
    if cutoff=='bonferroni':
        df=df[df[pcol]<0.05/len(df)]
    elif cutoff=='FDR':
        df['pval_FDR']=statsmodels.stats.multitest.fdrcorrection(df[pcol],alpha=0.05,method='indep',is_sorted=False)[1]
        df=df[df['pval_FDR']<0.05]
    else:
        print('cutoff not defined/custom- using all genes ')
        df=df
    print(df.head())
    #gene_ls=list(set(df[gene_col]))
    #return(gene_ls)
    return(df)

In [11]:
z_score, Fnew_score, Fnew_rand_score = netprop_zscore.calculate_heat_zscores(w_double_prime, pc_nodes, 
                                                            dict(interactome.degree), 
                                                            data, num_reps=1000,
                                                            minimum_bin_size=100)
trait_name=trait_name
z_score.to_csv('network_scores/'+trait_name+'_zscore.tsv',sep='\t',header=False)
Fnew_score.to_csv('network_scores/'+trait_name+'_heats.tsv',sep='\t',header=False)
pd.DataFrame(Fnew_rand_score, columns=z_score.index).to_csv('network_scores/'+trait_name+'_randheats.tsv',sep='\t')
print(str(trait_name+'_zscore.tsv'))
print(str('network_scores/'+trait_name+'_zscore.tsv'))

  0%|          | 0/1000 [00:00<?, ?it/s]

ADH1C_zscore.tsv
network_scores/ADH1C_zscore.tsv


# rare gene data analysis

In [21]:
os.getcwd()

'/tscc/projects/ps-palmer/brittany/rare_common_alcohol'

In [127]:
datasets=pd.read_csv('rare_datasets_prepub.csv')
runsets=datasets
runsets=runsets.reset_index()

In [128]:
runsets

,index,label,cutoff used,seed_path,delim,zscore_file,zscore_path,seed_gene_name,phenotype_group
0,0,alcoholintake_FDR_25,all tests FDR <0.25,input_files/rare_variant_genebass/alcohol_inta...,comma,alcoholintake_fdr_25_zscore.tsv,calculated_values/network_scores/alcoholintake...,Gene Name,alcohol
1,1,rare_neale_20153_irnt_FDR_25,all tests FDR <0.25,input_files/rare_variant_genebass/20153_irnt/2...,tab,rare_neale_20153_irnt_fdr_25_zscore.tsv,calculated_values/network_scores/rare_neale_20...,Gene Name,control
2,2,rare_neale_20016_FDR_25,all tests FDR <0.25,input_files/rare_variant_genebass/20016/20016_...,tab,rare_neale_20016_fdr_25_zscore.tsv,calculated_values/network_scores/rare_neale_20...,Gene Name,control
3,3,rare_neale_4194_FDR_25,all tests FDR <0.25,input_files/rare_variant_genebass/4194/4194_25...,tab,rare_neale_4194_fdr_25_zscore.tsv,calculated_values/network_scores/rare_neale_41...,Gene Name,control
4,4,rare_neale_78_FDR_25,all tests FDR <0.25,input_files/rare_variant_genebass/78/78_25FDR.tsv,tab,rare_neale_78_fdr_25_zscore.tsv,calculated_values/network_scores/rare_neale_78...,Gene Name,control
5,5,rare_neale_C50_FDR_25,all tests FDR <0.25,input_files/rare_variant_genebass/C50/C50_25FD...,tab,rare_neale_c50_fdr_25_zscore.tsv,calculated_values/network_scores/rare_neale_c5...,Gene Name,control
6,6,rare_neale_C44_FDR_25,all tests FDR <0.25,input_files/rare_variant_genebass/C44/C44_25FD...,tab,rare_neale_c44_fdr_25_zscore.tsv,calculated_values/network_scores/rare_neale_c4...,Gene Name,control
7,7,rare_neale_100016_FDR_25,all tests FDR <0.25,input_files/rare_variant_genebass/100016/10001...,tab,rare_neale_100016_fdr_25_zscore.tsv,calculated_values/network_scores/rare_neale_10...,Gene Name,control
8,8,rare_strin_allcut_alcoholintake,burden bonferroni < 0.05 in the whole table of...,input_files/rare_variant_genebass/alcohol_inta...,comma,rare_strin_allcut_alcoholintake_zscore.tsv,calculated_values/network_scores/rare_strin_al...,gene_symbol,alcohol


In [129]:
for row in range(len(runsets)):
    print('processing '+runsets['label'][row])
    run_net_prop(runsets['seed_path'][row], runsets['label'][row],'0',runsets['seed_gene_name'][row],
                 runsets['delim'][row],'no_cutoff',
                interactome_name)

processing alcoholintake_FDR_25
cutoff not defined/custom- using all genes 
    Gene Name          Gene Id  P-Value SKATO  P-Value Burden  P-Value SKAT  \
0       MMEL1  ENSG00000142606       0.000037        0.000017      0.002806   
1        MTOR  ENSG00000198793       0.000112        0.000036      0.048810   
2  AC118553.2  ENSG00000283761       0.000138        0.000128      0.000453   
3      KIF21B  ENSG00000116852       0.000063        0.000141      0.000477   
4       KDM5B  ENSG00000117139       0.000043        0.000023      0.010125   

   BETA Burden   Burden Set  Chrom : Position Chrom   Position  burden_FDR  \
0     0.001404  missense|LC      1.002587e+09     1    2586658    0.121878   
1    -0.018638         pLoF      1.011107e+09     1   11106515    0.149029   
2    -0.005127   synonymous      1.099970e+09     1   99970489    0.266147   
3    -0.014921         pLoF      1.200974e+09     1  200973806    0.273031   
4    -0.010240         pLoF      1.202729e+09     1  202728

AttributeError: 'str' object has no attribute 'degree'